In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import re
from scipy.signal import butter, filtfilt


##### Gathering data

In [ ]:
mmmp_dir = './Control'
standing_control = glob(os.path.join(mmmp_dir, '**/Standing Upright*_C[0-9]*.tsv'), recursive=True)

mmmp_dir = './PWP'
standing_pwp = glob(os.path.join(mmmp_dir, '**/Standing Upright*_P[0-9]*.tsv'), recursive=True)

In [ ]:
# Data passes thorugh low-pass butterworth filter of 4th order
def butter_lowpass_filter(data, cutoff_frequency=10, sampling_rate=100, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data, axis=0, padlen=0)
    return filtered_data

##### Reading file function

In [ ]:
NAN_special = -999999

def read_file(file):
    # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(file,sep = '\t',skiprows=11)
    df = df.fillna(NAN_special)
    df.columns = df.columns.str.strip() #eliminating unecesary spaces in column titles
    time =  df.loc[:, "Time"].to_numpy()
    
   
    marker_list = ['RSHO X', 'RSHO Y', 'RSHO Z', 'LSHO X', 'LSHO Y','LSHO Z', 
                   'LFHD X', 'LFHD Y', 'LFHD Z', 'RFHD X', 'RFHD Y', 'RFHD Z',
                   'STRN X', 'STRN Y', 'STRN Z',
                   'LASI X', 'LASI Y', 'LASI Z', 'RASI X', 'RASI Y', 'RASI Z',
                   'LTOE X', 'LTOE Y', 'LTOE Z','RTOE X', 'RTOE Y', 'RTOE Z',
                  'LANK X', 'LANK Y', 'LANK Z','RANK X', 'RANK Y', 'RANK Z',
                  'LHEE X', 'LHEE Y', 'LHEE Z','RHEE X', 'RHEE Y', 'RHEE Z',
                  'LKNE X', 'LKNE Y', 'LKNE Z','RKNE X', 'RKNE Y', 'RKNE Z']

    # create empty arrays for each marker
    RSHO = np.zeros((len(time), 3))
    LSHO = np.zeros((len(time), 3))
    LSTC = np.zeros((len(time), 3))
    RBAK = np.zeros((len(time), 3))
    LFHD = np.zeros((len(time), 3))
    RFHD = np.zeros((len(time), 3))
    LBHD = np.zeros((len(time), 3))
    RBHD = np.zeros((len(time), 3))
    STRN = np.zeros((len(time), 3))
    LASI = np.zeros((len(time), 3))
    RASI = np.zeros((len(time), 3))
    LPSI = np.zeros((len(time), 3))
    RPSI = np.zeros((len(time), 3))
    RTOE = np.zeros((len(time), 3))
    LTOE = np.zeros((len(time), 3))
    LANK = np.zeros((len(time), 3))
    RANK = np.zeros((len(time), 3))
    LHEE = np.zeros((len(time), 3))
    RHEE = np.zeros((len(time), 3))
    LKNE = np.zeros((len(time), 3))
    RKNE = np.zeros((len(time), 3))
    
    flag = False # this flAg will help us identify the files that are incomplete later 
    missing_markers = [] # this is a list to collect the missing markers in the file
    
    for col_name in marker_list:
        if col_name in df.columns:
            
            if col_name.startswith('RSHO'):
                RSHO = df.loc[:, ['RSHO X', 'RSHO Y', 'RSHO Z']].to_numpy()
                RSHO = butter_lowpass_filter(RSHO)
            elif col_name.startswith('LSHO'):
                LSHO = df.loc[:, ['LSHO X', 'LSHO Y', 'LSHO Z']].to_numpy()
                LSHO = butter_lowpass_filter(LSHO)
            elif col_name.startswith('LFHD'):
                LFHD = df.loc[:, ['LFHD X', 'LFHD Y', 'LFHD Z']].to_numpy()
                LFHD = butter_lowpass_filter(LFHD)
            elif col_name.startswith('RFHD'):
                RFHD = df.loc[:, ['RFHD X', 'RFHD Y', 'RFHD Z']].to_numpy()
                RFHD = butter_lowpass_filter(RFHD)
            elif col_name.startswith('STRN'):
                STRN = df.loc[:, ['STRN X', 'STRN Y', 'STRN Z']].to_numpy()
                STRN = butter_lowpass_filter(STRN)
            elif col_name.startswith('LASI'):
                LASI = df.loc[:, ['LASI X', 'LASI Y', 'LASI Z']].to_numpy()
                LASI = butter_lowpass_filter(LASI)
            elif col_name.startswith('RASI'):
                RASI = df.loc[:, ['RASI X', 'RASI Y', 'RASI Z']].to_numpy()
                RASI = butter_lowpass_filter(RASI)
            elif col_name.startswith('RTOE'):
                RTOE = df.loc[:, ['RTOE X', 'RTOE Y', 'RTOE Z']].to_numpy()
                RTOE = butter_lowpass_filter(RTOE)
            elif col_name.startswith('LTOE'):
                LTOE = df.loc[:, ['LTOE X', 'LTOE Y', 'LTOE Z']].to_numpy()
                LTOE = butter_lowpass_filter(LTOE)
            elif col_name.startswith('LANK'):
                LANK = df.loc[:, ['LANK X', 'LANK Y', 'LANK Z']].to_numpy()
                LANK = butter_lowpass_filter(LANK)
            elif col_name.startswith('RANK'):
                RANK = df.loc[:, ['RANK X', 'RANK Y', 'RANK Z']].to_numpy()
                RANK = butter_lowpass_filter(RANK)
            elif col_name.startswith('LHEE'):
                LHEE = df.loc[:, ['LHEE X', 'LHEE Y', 'LHEE Z']].to_numpy()
                LHEE = butter_lowpass_filter(LHEE)
            elif col_name.startswith('RHEE'):
                RHEE = df.loc[:, ['RHEE X', 'RHEE Y', 'RHEE Z']].to_numpy()
                RHEE = butter_lowpass_filter(RHEE)
            elif col_name.startswith('LKNE'):
                LKNE = df.loc[:, ['LKNE X', 'LKNE Y', 'LKNE Z']].to_numpy()
                LKNE = butter_lowpass_filter(LKNE)
            elif col_name.startswith('RKNE'):
                RKNE = df.loc[:, ['RKNE X', 'RKNE Y', 'RKNE Z']].to_numpy()
                RKNE = butter_lowpass_filter(RKNE)
        else:
            flag = True
            missing_markers.append(col_name)
#             print('File: {} , does not have marker {}'.format(file, col_name))
    print(file,missing_markers)
    return flag,missing_markers,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE,LKNE,RKNE

In [ ]:
flag,missing_markers,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE,LKNE,RKNE = read_file(standing_control[2])

## Peak AP  inclination angle (yz plane)

    This angle is calculated as the deviation of the trunk in the ap plane wrt the initial position (beginning of recording)

In [ ]:
def calculate_peak_AP_inclination(left_shoulder,right_shoulder,sternum):
    """
    Calculate the peak anterior-posterior (AP) inclination of the trunk based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker.
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.

    Returns:
    - float: The peak AP inclination angle in degrees.
    """

    
    trunk = (left_shoulder + right_shoulder + sternum) /3
        # Calculate the inclination angle with respect to the yz plane for each vector
    inclination_angle = np.arctan2(trunk[:, 2], trunk[:, 1])
    inclination_angle_degrees = np.degrees(inclination_angle)
    inclination_angle_degrees-=inclination_angle_degrees[0]
    peak_AP_inclination = max(inclination_angle_degrees)

    return peak_AP_inclination #,inclination_angle_degrees




## Peak ML inclination angle (xz plane)

    The peak ML inclination is calculated with the xz coordiantes of the sternum marker and shoulders. The max inclination represents the max deviation of the trunk wrt the initial position (beginning of recording)

In [ ]:
def calculate_peak_ML_inclination(left_shoulder,right_shoulder,sternum):
    """
    Calculate the peak medial-lateral (ML) inclination of the trunk based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker.
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.

    Returns:
    - float: The peak ML inclination angle in degrees.
    """
    
    trunk = (left_shoulder + right_shoulder + sternum) /3
        # Calculate the inclination angle along the xz plane for each vector
    inclination_angle = np.arctan2(trunk[:, 2], trunk[:, 0])

    inclination_angle_degrees = np.degrees(inclination_angle)
    inclination_angle_degrees -= inclination_angle_degrees[0]
    peak_ML_inclination = max(np.abs(inclination_angle_degrees))
    
    return peak_ML_inclination #,inclination_angle_degrees


# peak_ML_inclination,inclination_angle_degrees = calculate_peak_ML_inclination(LSHO,RSHO,STRN)
# plt.plot(time,inclination_angle_degrees)
# print(peak_ML_inclination)

## RMS of pelvis acceleration

In [ ]:
def calculate_pelvis_acc_rms(left_hip, right_hip,time):
    """
    Calculate the root mean square (RMS) of the pelvic  acceleration based on the position of the two hip markers.

    Parameters:
    - left_hip (numpy.ndarray): Position vector representing the left hip marker.
    - right_hip (numpy.ndarray): Position vector representing the right hip marker.

    Returns:
    - float: The RMS of the pelvic acceleration.
    """
    # Normalize positions by subtracting initial positions
    left_hip_rel = left_hip - left_hip[0]
    right_hip_rel = right_hip - right_hip[0] 
    
    pelvis = (left_hip_rel+ right_hip_rel)/2
    
    #3D pelvis velocity
    pelvis_3Dvel = (np.diff(pelvis, axis=0) / np.diff(time)[:, np.newaxis] )/1000
   
    # Compute 3D acceleration
    pelvis_3Dacc = np.diff(pelvis_3Dvel, axis=0) / np.diff(time[1:])[:, np.newaxis] 
    
    mag_acc = np.linalg.norm(pelvis_3Dacc, axis=1)
    
    # Calculate RMS of acceleration
    acc_rms = np.sqrt(np.mean(mag_acc**2))
    return acc_rms#single value

acc_pelvis_rms = calculate_pelvis_acc_rms(LASI,RASI,time)

# plt.title('STRN acc and RMS value')
# plt.plot(time,mag_acc, label = 'acc',color='b')
# plt.axhline(acc_rms, label = 'rms acc',color='r')
# # plt.axhline(np.mean(mag_acc), label = 'rms acc',color='g')

# # plt.legend()

## RMS of trunk acceleration 

Calculation method from Palmieri et al. 2013

In [ ]:
def calculate_acc_rms(left_shoulder, right_shoulder, sternum, time):
    """
    Calculate the root mean square (RMS) of the trunk acceleration based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker (N_samples x 3).
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker (N_samples x 3).
    - sternum (numpy.ndarray): Position vector representing the sternum marker (N_samples x 3).
    - time (numpy.ndarray): Time vector (N_samples).

    Returns:
    - float: The RMS of the trunk acceleration.
    - numpy.ndarray: Magnitudes of the trunk acceleration at each time step.
    """
    # Normalize positions by subtracting initial positions
    left_shoulder_rel = left_shoulder - left_shoulder[0]
    right_shoulder_rel = right_shoulder - right_shoulder[0]
    sternum_rel = sternum - sternum[0]
    
    # Calculate the trunk position based on relative positions
    trunk = (left_shoulder_rel + right_shoulder_rel + sternum_rel) / 3

     # Calculate the trunk velocity
    trunk_3Dvel = (np.diff(trunk, axis=0) / np.diff(time)[:, np.newaxis] )/1000

    # Compute 3D acceleration
    
    trunk_3Dacc = np.diff(trunk_3Dvel, axis=0) / np.diff(time[1:])[:, np.newaxis] 

    # Calculate magnitude of acceleration
    mag_acc = np.linalg.norm(trunk_3Dacc, axis=1)
    
    # Calculate RMS of acceleration
    acc_rms = np.sqrt(np.mean(mag_acc**2))
    
    return acc_rms#, mag_acc  # Return RMS and magnitudes



## Mean trunk jerk

In [ ]:
def calculate_trunk_jerk(left_shoulder,right_shoulder,sternum,time):
    """
    Calculate the mean jerk of the trunk based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker.
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.

    Returns:
    - float: The mean jerk of the trunk.
    """
    # Normalize positions by subtracting initial positions
    left_shoulder_rel = left_shoulder - left_shoulder[0]
    right_shoulder_rel = right_shoulder - right_shoulder[0]
    sternum_rel = sternum - sternum[0]
        
    trunk = (left_shoulder_rel + right_shoulder_rel + sternum_rel) / 3

    # Calculate the trunk velocity
    trunk_3Dvel = (np.diff(trunk, axis=0) / np.diff(time)[:, np.newaxis]) /1000
    # Compute 3D acceleration
    trunk_3Dacc = np.diff(trunk_3Dvel, axis=0) / np.diff(time[1:])[:, np.newaxis]  
    # Compute 3D jerk 
    trunk_3Djerk = np.diff(trunk_3Dacc, axis=0) / np.diff(time[2:])[:, np.newaxis]  

    mean_trunk_jerk = np.mean(trunk_3Djerk)
    return mean_trunk_jerk#single value

mean_jerk = calculate_trunk_jerk(LSHO,RSHO,STRN,time)


## Lateral inclination in the frontal plane

    We calculate the lateral body inclination with the angle composed by the vectors of shoulders and hips in the frontal plane (xz coordinates of shoulder and hip markers) considering the first frame as the reference point. 

In [ ]:
# Data passes thorugh low-pass butterworth filter of 4th order
def butter_lowpass_filter_inclination(data, cutoff_frequency, sampling_rate, order=2):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data, axis=0, padlen=0)
    return filtered_data


def calculate_lateral_inclination(right_shoulder,left_shoulder,left_hip,right_hip, cutoff_frequency=5, sampling_rate=100):
    shoulders = right_shoulder-left_shoulder # the shoulder vector points from left to right
    hips = right_hip-left_hip # the hip vector points from left to right 
    
    # Apply Butterworth filter to raw data 
    filtered_shoulders = butter_lowpass_filter_inclination(shoulders, cutoff_frequency, sampling_rate)
    filtered_hips = butter_lowpass_filter_inclination(hips, cutoff_frequency, sampling_rate)

    # Calculate filtered vectors
    shoulder_vector = filtered_shoulders[:,2] - filtered_shoulders[:,0]
    hip_vector = filtered_hips[:,2] - filtered_hips[:,0]

    # Get the norm of vectors
    shoulder_unit_vector = shoulder_vector / np.linalg.norm(shoulder_vector)
    hip_unit_vector = hip_vector / np.linalg.norm(hip_vector)

    # Calculate dot product
    dot_product = np.dot(shoulder_unit_vector, hip_unit_vector)

    # Calculate angle in radians
    angle_rad = np.arccos(dot_product)

    # Convert angle to degrees
    angle_deg = np.degrees(angle_rad)

    # Determine direction of inclination (right or left)
    inclination_direction = np.sign(shoulder_vector[0] * hip_vector[2] - shoulder_vector[2] * hip_vector[0])

    # Apply direction to the angle
    angle_with_direction = inclination_direction * angle_deg

    return angle_with_direction



lateral_inclination = calculate_lateral_inclination(RSHO,LSHO,LASI,RASI)
print(f"Lateral Inclination with Filtering: {lateral_inclination} degrees")


# Storing data

In [ ]:
#Create the dataframes to store the body sway
df_standing_upright = pd.DataFrame(columns=['Subject ID','Group','Peak_ML_Inclination[deg]','Peak_AP_Inclination[deg]',
                                       'Pelvis_Acc_RMS','Trunk_Acc_RMS','Mean_trunk_jerk[m/s3]','Shoulder_hip_angle[deg]'])


In [ ]:
full_data = [standing_control,standing_pwp]
data_label = ['Control','PwP']

for i,group in enumerate(full_data):
    for file in group:
        type_subject = data_label[i]
#         print(type_subject)
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.tsv$', file).group(1)
            subject_id = 'C'+subject_id
        else: 
            subject_id = re.search(r'_P(\d+)\.tsv$', file).group(1)
            subject_id = 'P'+subject_id


               
        flag,missing_markers,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE,LKNE,RKNE = read_file(file)
        
        #Verify if the sternum marker is present, to calculate all trunk-related metrics

        if any(marker.startswith('STRN','LSHO','RSHO') for marker in missing_markers):
            print(subject_id + "One or more required markers are missing.")
            peak_AP = peak_ML = trunk_acc_rms = mean_trunk_jerk = np.nan
            
             #Verify of the hip markers are present, to calculate the pelvis acc rms 

            if any(marker.startswith('LASI','RASI') for marker in missing_markers):
                acc_pelvis_rms = angle_shoulders_hips = np.nan

        else:
            peak_AP = calculate_peak_AP_inclination(LSHO,RSHO,STRN)
            peak_ML = calculate_peak_ML_inclination(LSHO,RSHO,STRN)
            trunk_acc_rms = calculate_acc_rms(LSHO,RSHO,STRN,time)
            mean_trunk_jerk = calculate_trunk_jerk(LSHO,RSHO,STRN,time)
            acc_pelvis_rms = calculate_pelvis_acc_rms(LASI,RASI,time)
            angle_shoulders_hips = calculate_lateral_inclination(RSHO,LSHO,LASI,RASI)

        new_line = {'Subject ID':subject_id,'Group':type_subject,
                   'Peak_ML_Inclination[deg]':peak_ML,'Peak_AP_Inclination[deg]':peak_AP,
                   'Pelvis_Acc_RMS': acc_pelvis_rms,
                    'Trunk_Acc_RMS': trunk_acc_rms,
                    'Mean_trunk_jerk[m/s3]': mean_trunk_jerk,
                    'Shoulder_hip_angle[deg]': angle_shoulders_hips
                   
                   }

        df_standing_upright.loc[len(df_standing_upright)] = new_line

In [ ]:
df_standing_upright